# Project 2 - Source coding, data compression and channel coding

The goal of this second project is to apply some of the principles seen in the lectures about source coding, data compression and channel coding. We ask you to write a brief report (pdf format) collecting your answers to the different questions. All codes must be written in Python inside this Jupyter. Note that you can not change the content of locked cells or import any extra Python library than the ones already imported (numpy).

## Implementation

In this project, you will need to use implement source coding algorithms to answer several questions. Therefore, in this first part, you are asked to write several functions that implement two of the  algorithms seen in the theoretical lectures and one new algorithm described in the project statement. Remember that you need to fill in this Jupyter Notebook to answer these questions. Pay particular attention to the required input and output format of each function.

In [2]:
# [Locked Cell] You can not import any extra Python library in this Notebook.
import numpy as np

### Question 1
Implement a function that returns a binary Huffman code for a given probability distribution. Give the main steps of your implementation. Verify your code on Exercise 7 of the second exercise session (TP2), and report the output of your code for this example. Explain how to extend your function to generate a Huffman code of any (output) alphabet size. 


In [9]:
class Node:
    def __init__(self, symbol, prob):
        self.symbol = symbol
        self.prob = prob
        self.left = None
        self.right = None
        self.code = ''

def Huffman_code(probability_dict):
    """
    Create the Huffman code for given probabilities.
    
    Arguments:
    probability_dict: A dictionary where keys are symbols (characters or strings)
                      and values are the probability of the symbol as a float or double.
    
    Returns:
    codewords: A dictionary with the name and the corresponding codeword.
               Keys are symbols (characters or strings) and values are associated
               codeword as a character or a string.
    """
    nodes = [Node(symbol, prob) for symbol, prob in probability_dict.items()]
    while len(nodes) > 1:

        nodes.sort(key=lambda node: node.prob)
        
        # Nodes with fewer probs
        left = nodes.pop(0)
        right = nodes.pop(0)

        # Combine
        new_node = Node(None, left.prob + right.prob)
        new_node.left = left
        new_node.right = right

        nodes.append(new_node)

    # Recursive function to assign codes to leaves
    def assign_codes(node, prefix=''):
        if node.symbol is not None:
            return {node.symbol: prefix}
        else:
            codes = {}
            codes.update(assign_codes(node.left, prefix + '0'))
            codes.update(assign_codes(node.right, prefix + '1'))
            return codes

    root = nodes[0]
    codeword_dict = assign_codes(root)

    return codeword_dict

probability_dict = {"A": 0.2, "B": 0.2, "C": 0.2, "D": 0.2, "E": 0.2} # example?
Huffman_code(probability_dict)


{'C': '00', 'D': '01', 'E': '10', 'A': '110', 'B': '111'}

### Question 2

Given a sequence of symbols, implement a function that returns a dictionary and the encoded sequence using the on-line Lempel-Ziv algorithm (see State of the art in data compression, slide 50/53). Reproduce and report the example given in the course. 

In [322]:
def LZ_Online(sequence):
    """
    The on-line Lempel-Ziv algorithm given a sequence of symbols.
    
    Arguments:
    - sequence: Sequence of symbols in the string format

    Return:
    - dictionary: the computed dictionary in the form:
      keys: symbol as character or string
      values: associated codeword as a tuple (integer) and a binarized address with one appended symbol (character or string)
      Example: {'0': (0, ''), '1': (1, '0'), '10': (2, '01'), '00': (3, '010')}
    
    - encoded_sequence: the encoded sequence in the string format
    """
    dictionary = {'': (0, '0')}
    encoded_sequence = ""
    w = ""
    # The first entry is special, handle it separately
    if sequence[0] == '1':
        dictionary['1'] = (1, '1')
        encoded_sequence += "(,1)"
        index = 2  
    else:
        dictionary['0'] = (1, '0')
        encoded_sequence += "(,0)"
        index = 2 
        
    for c in sequence[1:]:
        wc = w + c
        if wc not in dictionary:
            address_bits = int(np.ceil(np.log2(len(dictionary))))
            prefix = format(dictionary[w][0], '0{}b'.format(address_bits)) 
            encoded_sequence += f"({prefix},{c})"
            
            bin_address = format(index, '0{}b'.format(address_bits))
            dictionary[wc] = (index, bin_address) 
            
            index += 1
            w = ""
        else:
            w = wc
            
    # Encode the last phrase if it's not empty
    if w:
        address_bits = int(np.ceil(np.log2(max(len(dictionary), 1))))
        prefix = format(dictionary[w][0]-1, '0{}b'.format(address_bits))
        encoded_sequence += f"({prefix},)"
    
    return dictionary, encoded_sequence

sequence = "1011010100010"
dictionary, encoded_sequence = LZ_Online(sequence)
print(f'dictionary: {dictionary}')
print(f'encoded sequence: {encoded_sequence}')


dictionary: {'': (0, '0'), '1': (1, '1'), '0': (2, '10'), '11': (3, '11'), '01': (4, '100'), '010': (5, '101'), '00': (6, '110'), '10': (7, '111')}
encoded sequence: (,1)(0,0)(01,1)(10,1)(100,0)(010,0)(001,0)


### Question 4

Implement a function that returns the encoded sequence using the LZ77 algorithm as described by the algorithm below given an input string and a sliding window size l. Reproduce the example given in Figure 2 with window_size=7.

In [323]:
def LZ77(sequence, window_size=7):
    """
    The Lempel-Ziv 77 algorithm given a sequence of symbols and the sliding window size.
    
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    - window_size : sliding window size as an integer
    
    Return:
    -------
    - encoded_sequence : the encoded sequence in the string format
    """
    encoded_sequence = []
    cursor = 0

    while cursor < len(sequence):
        # Look-ahead buffer
        buffer = sequence[cursor:]
        window = sequence[max(0, cursor-window_size):cursor]

        p = 0
        d = 0
        c = ''
        
        for length in range(1, len(buffer)+1):
            prefix = buffer[:length]
            position = window.rfind(prefix)
            if  position>= 0:
                d = len(window) - position
                p = length
                c = buffer[length] if length < len(buffer) else ''
            else:
                break
             
        # If no match found, just take the first character of the buffer
        if p == 0 and len(buffer) > 0:
            c = buffer[0]
        
        encoded_sequence.append((d, p, c))
        shift = p + 1 if p > 0 else 1
        cursor += shift
    
    encoded_sequence_str = ''.join(['({},{},{})'.format(d, p, c) for d, p, c in encoded_sequence])
    
    return encoded_sequence_str

test_sequence = "abracadabrad"
test_window_size = 7
encoded_result = LZ77(test_sequence, test_window_size)
print(f'Encoded result: {encoded_result}')

Encoded result: (0,0,a)(0,0,b)(0,0,r)(3,1,c)(2,1,d)(7,4,d)


In [208]:
# [Locked Cell] Evaluation of your functions by the examiner. 
# You don't have access to the evaluation, this will be done by the examiner.
# Therefore, this cell will return nothing for the students.
import os
if os.path.isfile("private_evaluation.py"):
    from private_evaluation import unit_tests
    unit_tests(Huffman_code, LZ_online, LZ77)

## Source coding and reversible (lossless) data compression


In [ ]:
# Write here your codes for questions 5 to 15 (you may delete this comment)

## Channel coding

In [ ]:
# Write here your codes for questions 16 to 21 (you may delete this comment)